# Title

text text remove this

## Introduction
some introduction here (mention aqhi here)

## Preliminary exploratory data analysis

In [1]:
# Load libraries, run before everything else
library(tidyverse)
library(repr)
library(tidymodels)
install.packages("con2aqi")
library(con2aqi)
library(zoo) # for moving averages

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [2]:
# Get weather + pollution data for the Aotizhongxin station in Beijing
download.file("https://raw.githubusercontent.com/DonkeyBlaster/dsci-100-2023w1-group43/main/PRSA_Data_Aotizhongxin_20130301-20170228.csv", "Aotizhongxin_data.csv")
air_quality_data <- read_csv("Aotizhongxin_data.csv") |>
    select(-station) |> # This just says "Aotizhongxin", no need to keep it around
    select(-No)  # This is a continuously increasing counter, we don't need it either
head(air_quality_data, 3)
tail(air_quality_data, 3)

Rows: 35064 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): wd, station
dbl (16): No, year, month, day, hour, PM2.5, PM10, SO2, NO2, CO, O3, TEMP, P...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
2013,3,1,0,4,4,4,7,300,77,-0.7,1023.0,-18.8,0,NNW,4.4
2013,3,1,1,8,8,4,7,300,77,-1.1,1023.2,-18.2,0,N,4.7
2013,3,1,2,7,7,5,10,300,73,-1.1,1023.5,-18.2,0,NNW,5.6


year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
2017,2,28,21,16,37,10,66,700,58,10.8,1014.2,-13.3,0,NW,1.1
2017,2,28,22,21,44,12,87,700,35,10.5,1014.4,-12.9,0,NNW,1.2
2017,2,28,23,19,31,10,79,600,42,8.6,1014.1,-15.9,0,NNE,1.3


As shown above by the preview of the data, there are observations for each hour from the start of March 2013 to the end of February 2017. AQI can be easily calculated with the "con2aqi" library (after wrangling). First, we can check for and remove any N/A values:

In [3]:
air_quality_data <- air_quality_data |> na.omit()

And additionally, we need to wrangle the pollutant units into ones the library understands. All existing data are in ug/m^3, and the library wants the following units:
| PM2.5  | PM10   | SO2 | NO2 | CO  | O3  |
|--------|--------|-----|-----|-----|-----|
| ug/m^3 | ug/m^3 | ppb | ppb | ppm | ppm |

In [4]:
R = 0.082057366080960  # Gas constant for litres, atmospheres, kelvin, mols.
SO2_molecular_weight = 64.07  # g/mol
NO2_molecular_weight = 46.01  # g/mol
CO_molecular_weight = 28.01  # g/mol
O3_molecular_weight = 48.00  # g/mol
air_quality_data <- air_quality_data |>
    # PV = nRT formula rearranged to V = RT/P, n=1.
    mutate(volume = R * (273.2 + TEMP) / (PRES/1013)) |>   # Convert temp to Kelvin, pressure to atmospheres
    mutate(so2_ppb = volume * SO2 / SO2_molecular_weight) |>
    mutate(no2_ppb = volume * NO2 / NO2_molecular_weight) |>
    # Multiply by div by 1000 for ppb -> ppm
    mutate(co_ppm = volume * CO / CO_molecular_weight / 1000) |>
    mutate(o3_ppm = volume * O3 / O3_molecular_weight / 1000)
head(air_quality_data, 3)
tail(air_quality_data, 3)

year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,⋯,PRES,DEWP,RAIN,wd,WSPM,volume,so2_ppb,no2_ppb,co_ppm,o3_ppm
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2013,3,1,0,4,4,4,7,300,77,⋯,1023.0,-18.8,0,NNW,4.4,22.14205,1.382366,3.368711,0.2371516,0.03551954
2013,3,1,1,8,8,4,7,300,77,⋯,1023.2,-18.2,0,N,4.7,22.10523,1.380067,3.363108,0.2367572,0.03546047
2013,3,1,2,7,7,5,10,300,73,⋯,1023.5,-18.2,0,NNW,5.6,22.09875,1.724579,4.803032,0.2366878,0.03360852


year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,⋯,PRES,DEWP,RAIN,wd,WSPM,volume,so2_ppb,no2_ppb,co_ppm,o3_ppm
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2017,2,28,21,16,37,10,66,700,58,⋯,1014.2,-13.3,0,NW,1.1,23.27672,3.633014,33.38977,0.5817102,0.02812603
2017,2,28,22,21,44,12,87,700,35,⋯,1014.4,-12.9,0,NNW,1.2,23.24755,4.354152,43.95863,0.5809812,0.01695134
2017,2,28,23,19,31,10,79,600,42,⋯,1014.1,-15.9,0,NNE,1.3,23.09868,3.605226,39.66086,0.4947951,0.02021135


Next, we need to calculate certain moving averages for the concentration values, as per [the specification](https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf). Each pollutant has a different period for the required moving averages, listed in this table:
| PM2.5    | PM10     | SO2      | NO2    | CO      | O3           |
|----------|----------|----------|--------|---------|--------------|
| 24 hours | 24 hours | 1 hour   | 1 hour | 8 hours | 1 or 8 hours |


In [5]:
air_quality_data <- air_quality_data |>
    mutate(pm2.5_24hour = zoo::rollmean(PM2.5, k = 24, fill = NA, align = "right")) |>
    mutate(pm10_24hour = zoo::rollmean(PM10, k = 24, fill = NA, align = "right")) |>
    mutate(co_8hour = zoo::rollmean(co_ppm, k = 8, fill = NA, align = "right")) |>
    mutate(o3_8hour = zoo::rollmean(o3_ppm, k = 8, fill = NA, align = "right"))  # For o3 specifically, con2aqi allows us to choose 1 or 8 hours.
    # We're using 8 hours as the 1-hour window does not allow for reporting of AQI values less than 101.
head(air_quality_data, 26)

year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,⋯,WSPM,volume,so2_ppb,no2_ppb,co_ppm,o3_ppm,pm2.5_24hour,pm10_24hour,co_8hour,o3_8hour
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2013,3,1,0,4,4,4,7,300,77,⋯,4.4,22.14205,1.382366,3.368711,0.2371516,0.03551954,NA,NA,NA,NA
2013,3,1,1,8,8,4,7,300,77,⋯,4.7,22.10523,1.380067,3.363108,0.2367572,0.03546047,NA,NA,NA,NA
2013,3,1,2,7,7,5,10,300,73,⋯,5.6,22.09875,1.724579,4.803032,0.2366878,0.03360852,NA,NA,NA,NA
2013,3,1,3,6,6,11,11,300,72,⋯,3.1,22.05284,3.786191,5.272359,0.2361961,0.03307926,NA,NA,NA,NA
2013,3,1,4,3,3,12,12,300,72,⋯,2.0,21.98913,4.118458,5.735049,0.2355137,0.03298370,NA,NA,NA,NA
2013,3,1,5,5,5,18,18,400,66,⋯,3.7,21.96435,6.170723,8.592876,0.3136644,0.03020098,NA,NA,NA,NA
2013,3,1,6,3,3,18,32,500,50,⋯,2.5,21.91270,6.156213,15.240303,0.3911585,0.02282573,NA,NA,NA,NA
2013,3,1,7,3,6,19,41,500,43,⋯,3.8,21.97441,6.516526,19.581630,0.3922601,0.01968541,NA,NA,0.2849237,0.03042045
2013,3,1,8,3,6,16,43,500,45,⋯,4.1,22.09260,5.517115,20.647289,0.3943699,0.02071181,NA,NA,0.3045759,0.02856948


We can finally calculate the AQI values for each of the pollutants.

In [ ]:
air_quality_data <- air_quality_data |>
    na.omit() |>  #  We will remove all rows with NA first.
    mutate(pm2.5_aqi = con2aqi(pollutant = "pm25", con = pm2.5_24hour)) |>
    mutate(pm10_aqi = con2aqi(pollutant = "pm10", con = pm10_24hour)) |>
    mutate(so2_aqi = con2aqi(pollutant = "so2", con = so2_ppb)) |>
    mutate(no2_aqi = con2aqi(pollutant = "no2", con = no2_ppb)) |>
    mutate(co_aqi = con2aqi(pollutant = "co", con = co_8hour)) |>
    mutate(o3_aqi = con2aqi(pollutant = "o3", con = o3_8hour, type = "8h"))
air_quality_data
    

Because AQI is reported daily as the highest of the individual pollutant AQIs, we can obtain one final AQI value per day.

In [ ]:
air_quality_data <- air_quality_data |>
    select(year, month, day, hour, TEMP, PRES, DEWP, RAIN, WSPM, pm2.5_aqi, pm10_aqi, so2_aqi, no2_aqi, co_aqi, o3_aqi) |>
    group_by(year, month, day) |>
    summarize(across(TEMP:WSPM, mean), across(pm2.5_aqi:o3_aqi, max)) |>
    rowwise()|>
    mutate(aqi = round(max(pm2.5_aqi:o3_aqi)))
head(air_quality_data, 3)
tail(air_quality_data, 3)
    

We can visualize this data to gain some insight into how certain predictor variables may be affecting overall AQI.

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 6)
scaled_data <- air_quality_data |>
    group_by(year, month, day) |>
    summarize(across(TEMP:aqi, mean)) |>
    mutate(across(TEMP:aqi, scale))  # Normalize values to make them easier to compare

ggplot(scaled_data, aes(x = TEMP, y = aqi)) +
    geom_point(alpha = 0.3) +
    labs(x = "Normalized Temperature", y = " Normalized AQI", title = "Normalized Temperature vs. Normalized AQI")
ggplot(scaled_data, aes(x = DEWP, y = aqi)) +
    geom_point(alpha = 0.3) +
    labs(x = "Normalized Dew Point", y = " Normalized AQI", title = "Normalized Dew Point vs. Normalized AQI")
ggplot(scaled_data, aes(x = PRES, y = aqi)) +
    geom_point(alpha = 0.3) +
    labs(x = "Normalized Air Pressure", y = " Normalized AQI", title = "Normalized Air Pressure vs. Normalized AQI")
ggplot(scaled_data, aes(x = RAIN, y = aqi)) +
    geom_point(alpha = 0.3) +
    labs(x = "Normalized Rain", y = " Normalized AQI", title = "Normalized Rain vs. Normalized AQI")
ggplot(scaled_data, aes(x = WSPM, y = aqi)) +
    geom_point(alpha = 0.3) +
    labs(x = "Normalized Wind Speed", y = " Normalized AQI", title = "Normalized Wind Speed vs. Normalized AQI")

The relationship strength in the graphs above indicates which variables may be more useful in doing AQI predictions in the future.